'''
Must-have features:
1.time
2.title length
3.body length (cleaned)
4.tags length
5.codes included
6.codes length
7.web links included
8.images included
9.code comments included
10.code comments length

Future features:
1.tags quality
2.user info.
'''

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
import random as rd
from textblob import TextBlob
import requests
import time
from random import randint

data=pd.read_csv("Answers.csv",encoding= 'iso-8859-1')

In [25]:
data.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,497,50,2008-08-02T16:56:53Z,469,4,<p>open up a terminal (Applications-&gt;Utilit...
1,518,153,2008-08-02T17:42:28Z,469,2,<p>I haven't been able to find anything that d...
2,536,161,2008-08-02T18:49:07Z,502,9,<p>You can use ImageMagick's convert utility f...
3,538,156,2008-08-02T18:56:56Z,535,23,<p>One possibility is Hudson. It's written in...
4,541,157,2008-08-02T19:06:40Z,535,20,"<p>We run <a href=""http://buildbot.net/trac"">B..."


In [26]:
### Count how many code blocks, code comments, web link and images included in the text.
### Treat it as text features.
from html.parser import HTMLParser
    
class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)
    
def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def count_code_blocks(s):
    
    code_blocks = re.findall(r"<code>([\w\W]*?)</code>",s)
    return(len(code_blocks))

def count_code_length(s):
    
    code_blocks = re.findall(r"<code>([\w\W]*?)</code>",s)
    codes=" ".join(code_blocks)
    codes = strip_tags(codes) #remove html tags
    codes = re.sub(r" +"," ",codes)
    codes = codes.strip(" ")
    
    return(len(codes))

def count_code_comments(s):
    
    code_blocks = re.findall(r"<code>([\w\W]*?)</code>",s)
    comments = re.findall(r"#([^<]*)\n",s)
    real_comments =[comments[i] for i in range(len(comments)) if comments[i] in  " ".join(code_blocks)]# extract comments in codes
    
    return(len(real_comments))

def count_comments_length(s):
    
    code_blocks = re.findall(r"<code>([\w\W]*?)</code>",s)
    comments = re.findall(r"#([^<]*)\n",s)
    real_comments =[comments[i] for i in range(len(comments)) if comments[i] in  " ".join(code_blocks)]# extract comments in codes
    comments=" ".join(real_comments)
    comments = strip_tags(comments) #remove html tags
    comments = re.sub(r" +"," ",comments)
    comments = comments.strip(" ")
    
    return(len(comments))

def count_url(s):
    
    url = re.findall(r'<a [\w\W]+>(.*)</a>',s)
    
    return(len(url))

def count_img(s):
    
    img = re.findall(r"<img([\w\W]*?)>",s)

    return(len(img))

    

In [27]:
text_features=data
text_features['code_blocks_count']= text_features['Body'].map(lambda x:count_code_blocks(x))
text_features['code_comments_count']= text_features['Body'].map(lambda x:count_code_comments(x))
text_features['url_count']= text_features['Body'].map(lambda x:count_url(x))
text_features['img_count']= text_features['Body'].map(lambda x:count_img(x))


In [28]:
text_features['code_length']= text_features['Body'].map(lambda x:count_code_length(x))
text_features['comments_length']= text_features['Body'].map(lambda x:count_comments_length(x))

In [29]:
text_features['Body'][17]

u'<p>As far as I can tell Python, up through 2.5, only supports hexadecimal &amp; octal literals.  I did find some discussions about adding binary to future versions but nothing definite.</p>'

In [30]:
text_features['url_count'][17]

0

In [31]:
text_features['comments_length'][17]

0

In [32]:
del text_features['Body']

In [ ]:
text_features.head()
text_features.to_csv('python_answers_features.csv', sep=',', encoding='iso-8859-1')

In [23]:
text_features.head()

,Id,OwnerUserId,CreationDate,Score,Title,code_blocks_count,code_comments_count,url_count,img_count,code_length,comments_length
0,469,147,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,0,0,0,0,0,0
1,502,147,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,0,0,1,0,0,0
2,535,154,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,0,0,0,0,0,0
3,594,116,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,0,0,0,0,0,0
4,683,199,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,1,0,0,0,57,0
